In [104]:
import pandas as pd

# Loading data
df = pd.read_csv('./habits.data',
                sep=";",
                na_values=["?"],
                index_col=False,
                header=0)

# Naming columns 

# Listing variable types
# df['jasen'].values.dtype
df
# 
# Deleting variables not needed
# df.drop(['jasen'], axis=1, inplace=True)

# Filling missing values

# print(df.index.size, end="\n\n")

# Calculating missing values
# print((df.isna()).sum(axis=0), end="\n\n")

# Deleting missing values
# print(df.dropna().index.size, end="\n\n")
df = df.apply(pd.to_numeric, errors='coerce')
print(df.sum(axis=0).sort_values(ascending=False), end="\n\n")


kohde     38007543.0
V32         341100.0
V70          94220.0
V1           79820.0
V33          56980.0
V68          31440.0
V5           21300.0
V27          14790.0
V21           7450.0
V7            7130.0
V69           5100.0
IKAL1         4544.0
V75           4130.0
V22           3290.0
H1g_A         2811.0
H1a_A         2356.0
H1b_A         2290.0
H1i_A         2160.0
H1h_A         1427.0
ASALUE        1176.0
sp            1132.0
pvknro        1113.0
jasen         1012.0
V40            610.0
dtype: float64



In [90]:
# Counting individual values in column 'jasen' and testing if errors
one, two, faults = 0, 0, 0
fault_values = []
count = len(df.jasen)

for value in df.jasen.values:
    if value == 1:
        one += 1
    elif value == 2:
        two += 1 
    else:
        print(value)
        fault_values.append(value)
        
if count != one + two:
    print("Error in counting")

print(f" 1's: {one}\n", f"2's: {two}\n", f"Total count: {count}\n", f"Fault values: {fault_values}")

 1's: 478
 2's: 267
 Total count: 745
 Fault values: []


In [109]:
# BEGIN: Preserve specific demographic and activity variables
columns_to_keep = ['kohde', 'jasen', 'pvknro', 'sp', 'IKAL1', 'ASALUE', 'V1', 'V21', 'V22', 'V5', 'H1a_A', 'H1b_A']
df = df[columns_to_keep]
print(df)
# END:

     kohde  jasen  pvknro  sp  IKAL1  ASALUE     V1   V21  V22    V5  H1a_A  \
0    50002      1       1   1      6     1.0    0.0   0.0  0.0   0.0    2.0   
1    50002      1       2   1      6     1.0  380.0   0.0  0.0  10.0    2.0   
2    50003      1       1   2      5     2.0    0.0  50.0  0.0  60.0    2.0   
3    50003      1       2   2      5     2.0    0.0  50.0  0.0  50.0    2.0   
4    50004      2       1   1      7     1.0  640.0   0.0  0.0   0.0    2.0   
..     ...    ...     ...  ..    ...     ...    ...   ...  ...   ...    ...   
740  51980      1       2   2      6     2.0  460.0   0.0  0.0  20.0    1.0   
741  51981      2       1   1      5     1.0    0.0   0.0  0.0   0.0    NaN   
742  51981      2       2   1      5     1.0    0.0  10.0  0.0   0.0    NaN   
743  51983      1       1   2      8     3.0    NaN   0.0  0.0  40.0    1.0   
744  51983      1       2   2      8     3.0    0.0   0.0  0.0  90.0    1.0   

     H1b_A  
0      2.0  
1      2.0  
2      1.0  

In [110]:
# Use apply to get value counts for each column
value_counts_df = df.apply(pd.Series.value_counts).T


# Print the resulting DataFrame
print(value_counts_df)


        0.0      1.0      2.0      3.0      4.0      5.0      6.0      \
kohde       NaN      NaN      NaN      NaN      NaN      NaN      NaN   
jasen       NaN    478.0    267.0      NaN      NaN      NaN      NaN   
pvknro      NaN    377.0    368.0      NaN      NaN      NaN      NaN   
sp          NaN    358.0    387.0      NaN      NaN      NaN      NaN   
IKAL1       NaN      NaN      NaN     42.0     95.0    124.0    162.0   
ASALUE      NaN    472.0    115.0    158.0      NaN      NaN      NaN   
V1        458.0      NaN      NaN      NaN      NaN      NaN      NaN   
V21       572.0      NaN      NaN      NaN      NaN      NaN      NaN   
V22       611.0      NaN      NaN      NaN      NaN      NaN      NaN   
V5        178.0      NaN      NaN      NaN      NaN      NaN      NaN   
H1a_A      11.0    326.0    355.0      NaN      NaN      NaN      NaN   
H1b_A      11.0    332.0    349.0      NaN      NaN      NaN      NaN   

        7.0      8.0      9.0      ...  51946.0  5

In [113]:
# Describes the data 
df.describe().round(2)

,kohde,jasen,pvknro,sp,IKAL1,ASALUE,V1,V21,V22,V5,H1a_A,H1b_A
count,745.00,745.00,745.00,745.00,745.00,745.00,655.00,658.00,657.00,658.00,701.00,701.00
mean,51016.84,1.36,1.49,1.52,6.10,1.58,121.86,11.32,5.01,32.37,3.36,3.27
std,587.44,0.48,0.50,0.50,1.59,0.82,209.23,44.58,26.27,34.85,20.07,17.48
min,50002.00,1.00,1.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,50518.00,1.00,1.00,1.00,5.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00
50%,51053.00,1.00,1.00,2.00,6.00,1.00,0.00,0.00,0.00,20.00,2.00,2.00
75%,51524.00,2.00,2.00,2.00,7.00,2.00,205.00,0.00,0.00,50.00,2.00,2.00
max,51983.00,2.00,2.00,2.00,9.00,3.00,1050.00,500.00,410.00,190.00,360.00,300.00


In [114]:
# Printing out first rows of the data
df.head()

,kohde,jasen,pvknro,sp,IKAL1,ASALUE,V1,V21,V22,V5,H1a_A,H1b_A
0,50002,1,1,1,6,1.0,0.0,0.0,0.0,0.0,2.0,2.0
1,50002,1,2,1,6,1.0,380.0,0.0,0.0,10.0,2.0,2.0
2,50003,1,1,2,5,2.0,0.0,50.0,0.0,60.0,2.0,1.0
3,50003,1,2,2,5,2.0,0.0,50.0,0.0,50.0,2.0,1.0
4,50004,2,1,1,7,1.0,640.0,0.0,0.0,0.0,2.0,1.0


In [115]:
# Printing out basic information about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745 entries, 0 to 744
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   kohde   745 non-null    int64  
 1   jasen   745 non-null    int64  
 2   pvknro  745 non-null    int64  
 3   sp      745 non-null    int64  
 4   IKAL1   745 non-null    int64  
 5   ASALUE  745 non-null    float64
 6   V1      655 non-null    float64
 7   V21     658 non-null    float64
 8   V22     657 non-null    float64
 9   V5      658 non-null    float64
 10  H1a_A   701 non-null    float64
 11  H1b_A   701 non-null    float64
dtypes: float64(7), int64(5)
memory usage: 70.0 KB
